In [ ]:
pip install GetOldTweets3

In [ ]:
pip install fastai --upgrade

     |████████████████████████████████| 358kB 2.8MB/s 
     |████████████████████████████████| 10.5MB 12.4MB/s 
ERROR: google-colab 1.0.0 has requirement pandas~=1.0.0; python_version >= "3.0", but you'll have pandas 1.1.1 which is incompatible.
  Found existing installation: pandas 1.0.5
    Uninstalling pandas-1.0.5:
      Successfully uninstalled pandas-1.0.5
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [ ]:
from fastai.text.all import *
import pandas as pd

In [ ]:
learn_inf = load_learner("/content/drive/My Drive/data/sentiment analysis/export (1).pkl")

In [ ]:
import GetOldTweets3 as got
import pandas as pd


user_name = "@metpoliceuk"
q='to:{}'.format(user_name)

In [ ]:
tweetCriteria = got.manager.TweetCriteria().setQuerySearch(q)\
                                           .setSince("2020-08-25")\
                                           .setUntil("2020-08-26")
tweets = got.manager.TweetManager.getTweets(tweetCriteria)

df = pd.DataFrame(tweets)

def get_text(tweet):
  return tweet.text

def get_username(tweet):
  return tweet.username

df["username"] = df[0].apply(get_username)
df["text"] = df[0].apply(get_text)

In [ ]:
bios_df = df.copy()
bios = bios_df.dropna(axis=0, subset=["username"])
bios = bios.drop_duplicates(subset=["username"])

In [ ]:
import tweepy
import logging
from tweepy import API

API_KEY
API_SECRET
ACCESS_TOKEN
ACCESS_TOKEN_SECRET 

auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)



def get_bio(user):
  list_of_bios = pd.DataFrame(columns=["Users","Bios"])
  i = 100
  while i < len(user):
    users = user[i-100:i]
    test = api.lookup_users(screen_names =users)
    for person in test:
      list_of_bios = list_of_bios.append({'Users' : person.screen_name , 'Bios' : person.description}, ignore_index=True)
    i = i + 100
  users = user[i-100:len(user)]
  test = api.lookup_users(screen_names =users)
  for person in test:
      list_of_bios = list_of_bios.append({'Users' : person.screen_name , 'Bios' : person.description}, ignore_index=True)
  return list_of_bios
 


list_of_new_bios = get_bio(df["username"].tolist())

In [ ]:
def get_label(row):
  return row[0]

list_of_new_bios["RW_prediction"] = list_of_new_bios["Bios"].apply(learn_inf.predict)

list_of_new_bios["RW_prediction"] = list_of_new_bios["RW_prediction"].apply(get_label)

In [ ]:
list_of_new_bios

,Users,Bios,RW_prediction
0,Humza_Khann,London 📍| Instagram: HumzaKhan_,False
1,PDF53,Retired Logistics Contractor;\nIroning my own shirts since '69;\nUnashamed to Back Britain; Back on here April 20. #AllLivesMatter\n#DefundTheBBC #mufc #brexit,False
2,katunar07,🇦🇱🇽🇰,False
3,claire_bates,"Journo, mummy, carer, Army wife and star of Sesame Street. In differing orders of importance. British but squatting in the States for a bit",False
4,ChrisWinker,,False
...,...,...,...
105,DJBURNS_was,"Follow me, I’ll follow back 🇬🇧 stop illegal migrants 💪 very right wing 👉 love 🇺🇸 used to be DJBURNS. All comments are my own.",True
106,SP92328669,"London hairdresser, voted Brexit. Don’t fund BBC, buy British. Daughter of immigrant, Britalian. Follow F1, love yoga and meditation. love sarcasm",True
107,ashishchakrabor,A licensed commercial manger with over 20 years’ experience in the licensed and hospitality industry. Love rum and coke.,False
108,Budgey_95,🇾🇪,False


In [ ]:
list_of_new_bios = list_of_new_bios.rename({"Users":"username"}, axis=1)

In [ ]:
df

,0,username,text
0,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad768aa20>,Humza_Khann,@metpoliceuk
1,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad768a908>,PDF53,@metpoliceuk @MPFed @MetCC The film ..now..! @DawnButlerBrent @Keir_Starmer
2,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad768af28>,PDF53,@metpoliceuk @MetCC Show the film. Now !
3,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad768a8d0>,katunar07,@metpoliceuk you’re needed
4,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad768a2b0>,claire_bates,@metpoliceuk got away extraordinarily lightly in this @BBCDocs investigation. Disappointingly lightweight.
...,...,...,...
115,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad7636c18>,DJBURNS_was,"Does the driving ban marry up with the sentence? Depending how long he serves, will he be able to drive when he gets out? The mind boggles"
116,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad7621978>,SP92328669,I hope the disqualification is starting after his sentence or it’s pointless.
117,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad7621ef0>,ashishchakrabor,"After watching all the police shows like interceptors and traffic cops, I feel the law lets the police down with the sentencing .the hard work and length they go to fight crimes. Must be frustrating. Stay safe"
118,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad7636eb8>,Budgey_95,@metpoliceuk fuck you all dirty racist basteds


In [ ]:
new_df = df.merge(list_of_new_bios, on=["username"])

In [ ]:
new_df

,0,username,text,Bios,RW_prediction
0,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad768aa20>,Humza_Khann,@metpoliceuk,London 📍| Instagram: HumzaKhan_,False
1,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad768a908>,PDF53,@metpoliceuk @MPFed @MetCC The film ..now..! @DawnButlerBrent @Keir_Starmer,Retired Logistics Contractor;\nIroning my own shirts since '69;\nUnashamed to Back Britain; Back on here April 20. #AllLivesMatter\n#DefundTheBBC #mufc #brexit,False
2,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad768af28>,PDF53,@metpoliceuk @MetCC Show the film. Now !,Retired Logistics Contractor;\nIroning my own shirts since '69;\nUnashamed to Back Britain; Back on here April 20. #AllLivesMatter\n#DefundTheBBC #mufc #brexit,False
3,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad768a8d0>,katunar07,@metpoliceuk you’re needed,🇦🇱🇽🇰,False
4,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad768a2b0>,claire_bates,@metpoliceuk got away extraordinarily lightly in this @BBCDocs investigation. Disappointingly lightweight.,"Journo, mummy, carer, Army wife and star of Sesame Street. In differing orders of importance. British but squatting in the States for a bit",False
...,...,...,...,...,...
119,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad7636c18>,DJBURNS_was,"Does the driving ban marry up with the sentence? Depending how long he serves, will he be able to drive when he gets out? The mind boggles","Follow me, I’ll follow back 🇬🇧 stop illegal migrants 💪 very right wing 👉 love 🇺🇸 used to be DJBURNS. All comments are my own.",True
120,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad7621978>,SP92328669,I hope the disqualification is starting after his sentence or it’s pointless.,"London hairdresser, voted Brexit. Don’t fund BBC, buy British. Daughter of immigrant, Britalian. Follow F1, love yoga and meditation. love sarcasm",True
121,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad7621ef0>,ashishchakrabor,"After watching all the police shows like interceptors and traffic cops, I feel the law lets the police down with the sentencing .the hard work and length they go to fight crimes. Must be frustrating. Stay safe",A licensed commercial manger with over 20 years’ experience in the licensed and hospitality industry. Love rum and coke.,False
122,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad7636eb8>,Budgey_95,@metpoliceuk fuck you all dirty racist basteds,🇾🇪,False


In [ ]:
from textblob import TextBlob
from wordcloud import WordCloud

In [ ]:
# Create a function to clean the tweets
def cleanTxt(text):
 text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
 text = re.sub('#', '', text) # Removing '#' hash tag
 text = re.sub('RT[\s]+', '', text) # Removing RT
 text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
 
 return text


# Clean the tweets
new_df['clean_text'] = new_df['text'].apply(cleanTxt)

# Show the cleaned tweets
new_df

,0,username,text,Bios,RW_prediction,clean_text
0,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad768aa20>,Humza_Khann,@metpoliceuk,London 📍| Instagram: HumzaKhan_,False,
1,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad768a908>,PDF53,@metpoliceuk @MPFed @MetCC The film ..now..! @DawnButlerBrent @Keir_Starmer,Retired Logistics Contractor;\nIroning my own shirts since '69;\nUnashamed to Back Britain; Back on here April 20. #AllLivesMatter\n#DefundTheBBC #mufc #brexit,False,The film ..now..! _Starmer
2,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad768af28>,PDF53,@metpoliceuk @MetCC Show the film. Now !,Retired Logistics Contractor;\nIroning my own shirts since '69;\nUnashamed to Back Britain; Back on here April 20. #AllLivesMatter\n#DefundTheBBC #mufc #brexit,False,Show the film. Now !
3,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad768a8d0>,katunar07,@metpoliceuk you’re needed,🇦🇱🇽🇰,False,you’re needed
4,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad768a2b0>,claire_bates,@metpoliceuk got away extraordinarily lightly in this @BBCDocs investigation. Disappointingly lightweight.,"Journo, mummy, carer, Army wife and star of Sesame Street. In differing orders of importance. British but squatting in the States for a bit",False,got away extraordinarily lightly in this investigation. Disappointingly lightweight.
...,...,...,...,...,...,...
119,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad7636c18>,DJBURNS_was,"Does the driving ban marry up with the sentence? Depending how long he serves, will he be able to drive when he gets out? The mind boggles","Follow me, I’ll follow back 🇬🇧 stop illegal migrants 💪 very right wing 👉 love 🇺🇸 used to be DJBURNS. All comments are my own.",True,"Does the driving ban marry up with the sentence? Depending how long he serves, will he be able to drive when he gets out? The mind boggles"
120,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad7621978>,SP92328669,I hope the disqualification is starting after his sentence or it’s pointless.,"London hairdresser, voted Brexit. Don’t fund BBC, buy British. Daughter of immigrant, Britalian. Follow F1, love yoga and meditation. love sarcasm",True,I hope the disqualification is starting after his sentence or it’s pointless.
121,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad7621ef0>,ashishchakrabor,"After watching all the police shows like interceptors and traffic cops, I feel the law lets the police down with the sentencing .the hard work and length they go to fight crimes. Must be frustrating. Stay safe",A licensed commercial manger with over 20 years’ experience in the licensed and hospitality industry. Love rum and coke.,False,"After watching all the police shows like interceptors and traffic cops, I feel the law lets the police down with the sentencing .the hard work and length they go to fight crimes. Must be frustrating. Stay safe"
122,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad7636eb8>,Budgey_95,@metpoliceuk fuck you all dirty racist basteds,🇾🇪,False,fuck you all dirty racist basteds


In [ ]:
# Create a function to get the subjectivity
def getSubjectivity(text):
   return TextBlob(text).sentiment.subjectivity

# Create a function to get the polarity
def getPolarity(text):
   return  TextBlob(text).sentiment.polarity


# Create two new columns 'Subjectivity' & 'Polarity'
new_df['Subjectivity'] = new_df['clean_text'].apply(getSubjectivity)
new_df['Polarity'] = new_df['clean_text'].apply(getPolarity)

# Show the new dataframe with columns 'Subjectivity' & 'Polarity'
new_df

,0,username,text,Bios,RW_prediction,clean_text,Subjectivity,Polarity
0,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad768aa20>,Humza_Khann,@metpoliceuk,London 📍| Instagram: HumzaKhan_,False,,0.000000,0.000000
1,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad768a908>,PDF53,@metpoliceuk @MPFed @MetCC The film ..now..! @DawnButlerBrent @Keir_Starmer,Retired Logistics Contractor;\nIroning my own shirts since '69;\nUnashamed to Back Britain; Back on here April 20. #AllLivesMatter\n#DefundTheBBC #mufc #brexit,False,The film ..now..! _Starmer,0.000000,0.000000
2,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad768af28>,PDF53,@metpoliceuk @MetCC Show the film. Now !,Retired Logistics Contractor;\nIroning my own shirts since '69;\nUnashamed to Back Britain; Back on here April 20. #AllLivesMatter\n#DefundTheBBC #mufc #brexit,False,Show the film. Now !,0.000000,0.000000
3,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad768a8d0>,katunar07,@metpoliceuk you’re needed,🇦🇱🇽🇰,False,you’re needed,0.000000,0.000000
4,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad768a2b0>,claire_bates,@metpoliceuk got away extraordinarily lightly in this @BBCDocs investigation. Disappointingly lightweight.,"Journo, mummy, carer, Army wife and star of Sesame Street. In differing orders of importance. British but squatting in the States for a bit",False,got away extraordinarily lightly in this investigation. Disappointingly lightweight.,0.700000,-0.100000
...,...,...,...,...,...,...,...,...
119,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad7636c18>,DJBURNS_was,"Does the driving ban marry up with the sentence? Depending how long he serves, will he be able to drive when he gets out? The mind boggles","Follow me, I’ll follow back 🇬🇧 stop illegal migrants 💪 very right wing 👉 love 🇺🇸 used to be DJBURNS. All comments are my own.",True,"Does the driving ban marry up with the sentence? Depending how long he serves, will he be able to drive when he gets out? The mind boggles",0.512500,0.225000
120,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad7621978>,SP92328669,I hope the disqualification is starting after his sentence or it’s pointless.,"London hairdresser, voted Brexit. Don’t fund BBC, buy British. Daughter of immigrant, Britalian. Follow F1, love yoga and meditation. love sarcasm",True,I hope the disqualification is starting after his sentence or it’s pointless.,0.300000,-0.125000
121,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad7621ef0>,ashishchakrabor,"After watching all the police shows like interceptors and traffic cops, I feel the law lets the police down with the sentencing .the hard work and length they go to fight crimes. Must be frustrating. Stay safe",A licensed commercial manger with over 20 years’ experience in the licensed and hospitality industry. Love rum and coke.,False,"After watching all the police shows like interceptors and traffic cops, I feel the law lets the police down with the sentencing .the hard work and length they go to fight crimes. Must be frustrating. Stay safe",0.557639,-0.086806
122,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8ad7636eb8>,Budgey_95,@metpoliceuk fuck you all dirty racist basteds,🇾🇪,False,fuck you all dirty racist basteds,0.700000,-0.500000


In [ ]:
right_wing_mask = new_df["RW_prediction"] == "True"
right_wing = new_df[right_wing_mask]

not_right_wing_mask = new_df["RW_prediction"] == "False"
not_right_wing = new_df[not_right_wing_mask]

In [ ]:
import plotly.express as px
fig2 = px.scatter(new_df, x="Subjectivity", y="Polarity", color="RW_prediction", hover_data=["Bios"])
fig2.update_xaxes(range=[0,1])
fig2.update_yaxes(range=[-1, 1])
fig2.show()

In [ ]:
grouped_by_RW = new_df.groupby("RW_prediction").agg(
    count= ("text", len),
    Subjectivity = ("Subjectivity", np.mean),
    Polarity = ("Polarity", np.mean)
)

In [ ]:
grouped_by_RW

,count,Subjectivity,Polarity
RW_prediction,,,
False,112,0.319138,0.002931
True,12,0.255324,-0.018519


In [ ]:
import plotly.express as px
fig1 = px.bar(grouped_by_RW, y="count", color=grouped_by_RW.index)
fig1.show()

In [ ]:
round(grouped_by_RW["count"].iloc[1] / grouped_by_RW["count"].iloc[0],3) * 100

10.7

In [ ]:
def run_process(user_name, date_from, date_to):

  q='to:{}'.format(user_name)

  tweetCriteria = got.manager.TweetCriteria().setQuerySearch(q)\
                                            .setSince(date_from)\
                                            .setUntil(date_to)
  tweets = got.manager.TweetManager.getTweets(tweetCriteria)

  df = pd.DataFrame(tweets)

  def get_text(tweet):
    return tweet.text

  def get_username(tweet):
    return tweet.username

  df["username"] = df[0].apply(get_username)
  df["text"] = df[0].apply(get_text)

  bios_df = df.copy()
  bios = bios_df.dropna(axis=0, subset=["username"])
  bios = bios.drop_duplicates(subset=["username"])

  auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
  auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

  api = tweepy.API(auth)

  list_of_new_bios = get_bio(df["username"].tolist())

  list_of_new_bios["RW_prediction"] = list_of_new_bios["Bios"].apply(learn_inf.predict)

  list_of_new_bios["RW_prediction"] = list_of_new_bios["RW_prediction"].apply(get_label)

  list_of_new_bios = list_of_new_bios.rename({"Users":"username"}, axis=1)

  new_df = df.merge(list_of_new_bios, on=["username"])



  # Clean the tweets
  new_df['clean_text'] = new_df['text'].apply(cleanTxt)

    # Create two new columns 'Subjectivity' & 'Polarity'
  new_df['Subjectivity'] = new_df['clean_text'].apply(getSubjectivity)
  new_df['Polarity'] = new_df['clean_text'].apply(getPolarity)

  right_wing_mask = new_df["RW_prediction"] == "True"
  right_wing = new_df[right_wing_mask]

  not_right_wing_mask = new_df["RW_prediction"] == "False"
  not_right_wing = new_df[not_right_wing_mask]

  fig2 = px.scatter(new_df, x="Subjectivity", y="Polarity", color="RW_prediction", hover_data=["Bios"])
  fig2.update_xaxes(range=[-0.1,1.1])
  fig2.update_yaxes(range=[-1.1, 1.1])

  grouped_by_RW = new_df.groupby("RW_prediction").agg(
    count= ("text", len),
    Subjectivity = ("Subjectivity", np.mean),
    Polarity = ("Polarity", np.mean))
  
  fig1 = px.bar(grouped_by_RW, y="count", color=grouped_by_RW.index)

  final = round(grouped_by_RW["count"].iloc[1] / grouped_by_RW["count"].iloc[0],3) * 100

  return grouped_by_RW, fig1, fig2, final
  

# Right Wing Follower Test

This tool is designed to check how many explicitly right wing users interact with a twitter account on a certain day, and how they differ to the rest of the interactions.

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from ipywidgets import *

In [ ]:
username_select =widgets.Text(
    value='@metpoliceuk',
    placeholder='Type something',
    description='Username:',
    disabled=False
)

In [ ]:
date_from_widget =widgets.Text(
    value='2020-08-27',
    placeholder='Type something',
    description='Date From:',
    disabled=False
)

In [ ]:
date_to_widget =widgets.Text(
    value='2020-08-28',
    placeholder='Type something',
    description='Date To:',
    disabled=False
)

In [ ]:
def show_graph2(change):
  with out_df:
    print("analysing..")
  results = run_process(username_select.value, date_from_widget.value, date_to_widget.value)
  out_df.clear_output()
  out_analysis.clear_output()
  out_graph1.clear_output()
  out_graph2.clear_output()
  with out_df:
    print(str(results[3]) + "% RW interactions")
  with out_analysis:
    print(results[0])
  with out_graph1:
    results[1].show()
  with out_graph2:
    results[2].show()
  


In [ ]:
analyse_button = widgets.Button(description='Analyse')

analyse_button.on_click(show_graph2)

In [ ]:
out_df = widgets.Output(layout={'border': '1px solid black'})
out_analysis = widgets.Output(layout={'border': '1px solid black'})
out_graph1 = widgets.Output(layout={'border': '1px solid black'})
out_graph2 = widgets.Output(layout={'border': '1px solid black'})

In [533]:
VBox([widgets.Label('Tweet Monitor'),
      username_select, date_from_widget, date_to_widget, analyse_button, out_df, out_analysis, out_graph1, out_graph2])

8.3% RW interactions
               count  Subjectivity  Polarity
RW_prediction                               
False            133      0.289162  0.025225
True              11      0.247890  0.055952


In [ ]:

def run_test(change):
  input = str(test_box.value)
  results = learn_inf.predict(input)
  test_out.clear_output()
  with test_out:
    print("Prediction for " + input + " is... "+results[0])

test_button = widgets.Button(description='Test')

test_button.on_click(run_test)


In [ ]:
test_out = widgets.Output(layout={'border': '1px solid black'})


test_box =widgets.Text(
    value='Enter text test here',
    placeholder='Type something',
    description='String:',
    disabled=False
)

In [ ]:
learn_inf.predict("not woke")

('False', tensor(0), tensor([0.8779, 0.1221]))

In [ ]:
VBox([widgets.Label('Output Test'),
      test_box, test_button, test_out])

In [ ]:

results = run_process("@metpoliceuk","2020-08-26", "2020-08-27")

In [ ]:
round(results[0]["count"].iloc[1] / results[0]["count"].iloc[0],3) * 100

14.7

In [ ]:
date_from = "2020-08-27"
date_to = "2020-08-28"
user_name="@metpoliceuk"
q='to:{}'.format(user_name)

tweetCriteria = got.manager.TweetCriteria().setQuerySearch(q)\
                                          .setSince(date_from)\
                                          .setUntil(date_to)
tweets = got.manager.TweetManager.getTweets(tweetCriteria)

In [ ]:
tweets

In [ ]:

df = pd.DataFrame(tweets)

def get_text(tweet):
  return tweet.text

def get_username(tweet):
  return tweet.username

df["username"] = df[0].apply(get_username)
df["text"] = df[0].apply(get_text)

In [ ]:
df

,0,username,text
0,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8acb22f9b0>,Samsson31363375,@metpolice
1,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8acb22fa90>,KarmaKarlz,@metpolice UK #Police What stops you looking like this #stopandsearch
2,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8acb22f978>,ATOMONOUS1,@MetPolice @MetPolice @MotabilityOps @RSAMotability @dwppressoffice #WW3#Totalitarianism
3,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8acdc87e48>,Samsson31363375,@metpolice @pritipatel @BorisJohnson
4,<GetOldTweets3.models.Tweet.Tweet object at 0x7f8acdc87940>,_jannatahmed,@metpolice he’s got their photo in his camera roll
